In [1]:
import numpy as np
import rdkit
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem.Pharm3D import Pharmacophore, EmbedLib
from rdkit.Chem import ChemicalFeatures
from rdkit.Chem.rdMolAlign import AlignMol
from rdkit.Chem import rdFMCS
from rdkit.Chem import fmcs

#Drawing stuff
from matplotlib.colors import ColorConverter
from rdkit.Chem.Draw import rdMolDraw2D
from PIL import Image
import io
from collections import defaultdict

%run /home/venkata/python/python_libraries/extras/Math.ipynb

Loaded Math library


In [4]:
COLORSEQ=[ColorConverter().to_rgb(col) for col in ("red","blue","cyan","green","grey","magenta","orange")]
TRANSP_VALUE=0.5
COLORSEQ_TRANSP=[(v[0],v[1],v[2],TRANSP_VALUE) for v in COLORSEQ]
def reassignTransparencies(val=0.5):
    global COLORSEQ_TRANSP,TRANSP_VALUE
    COLORSEQ_TRANSP=[(v[0],v[1],v[2],val) for v in COLORSEQ]
    TRANSP_VALUE=val

In [2]:
def findMCS(mols,mol2=None,ringMatchesRingOnly=True):
    if mol2 is not None:
        mols=[mols,mol2]
    return Chem.rdFMCS.FindMCS(mols,matchValences=False,timeout=10800,bondCompare=rdFMCS.BondCompare.CompareOrderExact,ringMatchesRingOnly=ringMatchesRingOnly)

In [3]:
DEFAULT_FEATURES=AllChem.BuildFeatureFactory(rdkit.__path__[0]+'/Data/BaseFeatures.fdef')
with open(rdkit.__path__[0]+'/Data/BaseFeatures.fdef') as f:
    featDef = f.read()
CHEMICAL_FEATURES=ChemicalFeatures.BuildFeatureFactoryFromString(featDef)
featDef=None

In [4]:
def alignMolsIn3D(mol1,mols):
    if type(mols)==type(mol1): mols=[mols]
    rmsds=[]
    for mol2 in mols:
        substructure = findMCS(mol1,mol2)
        substructure = Chem.MolFromSmarts(substructure.smartsString)

        AllChem.EmbedMolecule(mol1)
        AllChem.EmbedMolecule(mol2)

        rmsds.append(AllChem.AlignMol(mol2, mol1, atomMap=list(zip(mol2.GetSubstructMatch(substructure), mol1.GetSubstructMatch(substructure))),maxIters=500))
    return rmsds

def identifyPharmacophores(mol): return CHEMICAL_FEATURES.GetFeaturesForMol(mol)

In [5]:
def sortFeaturesByFamily(feats):
    ret=dict()
    for f in feats:
        fn=f.GetFamily()
        if fn in ret: ret[fn].append(f)
        else: ret[fn]=[f]
    return ret
def findCommonPharmacophores(mol1,mol2,nearCutoff=1.0): #Nearness cutoff in 3D space (Angstroms)
    alignMolsIn3D(mol1,[mol2])
    featLists=[identifyPharmacophores(m) for m in [mol1,mol2]]
    featDicts=[sortFeaturesByFamily(fl) for fl in featLists]
    featdict1=featDicts[0]
    featdict2=featDicts[1]
    
    allowed_matches=[]
    for featfam in featdict1:
        if featfam not in featdict2: continue
        for f1 in featdict1[featfam]:
            for f2 in featdict2[featfam]:
                dist = (f1.GetPos() - f2.GetPos()).Length()
                if dist<nearCutoff:
                    allowed_matches.append((f1,f2))
    return allowed_matches,featLists[0],featLists[1]

In [6]:
def uniquelyMatchPharmacophores(allowed_matches):
    ids=[(set(f1.GetAtomIds()),set(f2.GetAtomIds())) for (f1,f2) in allowed_matches]
    matched_num=0
    
    clashes1=[]
    clashes2=[]
    clean1=set([])
    clean2=set([])
    final_matchings=[]
    failed_matchings=[]
    for i,(s1,s2) in enumerate(ids):
        clear1=True
        clear2=True
        for j,(sx1,sx2) in enumerate(ids):
            if i==j: continue
            if len(sx1.intersection(s1)):
                clear1=False
                clashes1.append((s1,sx1,s2,sx2))
            if len(sx2.intersection(s2)):
                clear2=False
                clashes2.append((s1,sx1,s2,sx2))
        if clear1: clean1=clean1.union(s1)
        if clear2: clean2=clean2.union(s2)
        if clear1 and clear2:
            final_matchings.append((s1,s2))
        else:
            failed_matchings.append((s1,s2))
    matching_subset=[]
    score=0
    for ss in enumerateNonintersectingSetCombinations(failed_matchings,intersectfn=lambda st1,st2: len(st1[0].intersection(st2[0]))+len(st1[1].intersection(st2[1]))!=0):
        ms=len(setUnion([s[0] for s in ss]))
        ms+=len(setUnion([s[1] for s in ss]))
        if ms>score:
            score=ms
            matching_subset=ss
    return final_matchings+matching_subset

In [6]:
def multipleHighlightDraw(mol,highlightKey,bondHighlight=dict()):
    testhg =  defaultdict(list)
    for idx in highlightKey:
        testhg[idx].append(highlightKey[idx])
    
    bondhg = defaultdict(list)
    for idx in bondHighlight:
        bondhg[idx].append(bondHighlight[idx])

    drawer = rdMolDraw2D.MolDraw2DCairo(350,300)
    drawer.drawOptions().highlightBondWidthMultiplier=20
    drawer.drawOptions().clearBackground = False
    drawer.DrawMoleculeWithHighlights(mol,"",dict(testhg),dict(bondhg),dict(),{})
    drawer.FinishDrawing()
    bio = io.BytesIO(drawer.GetDrawingText())
    return Image.open(bio)
    
    
def highlightPairedMatchings(mol1,mol2,matchings,idx=0,transp=0.6): #Use idx=1 to switch to mol2
    mol=mol1 if idx==0 else mol2
    if idx>0: idx=1
    tval=TRANSP_VALUE
    reassignTransparencies(transp)
    fragcols=dict()
    fragbcols=dict()
    colid=0
    for p in matchings:
        myset=p[idx]
        for at in myset:
            fragcols[at]=COLORSEQ_TRANSP[colid]
            for at2 in myset:
                if at<at2: continue
                bt=mol.GetBondBetweenAtoms(at,at2)
                if bt is not None: fragbcols[bt.GetIdx()]=COLORSEQ_TRANSP[colid]
        colid+=1
        if colid>=len(COLORSEQ_TRANSP): colid=0
    ret=multipleHighlightDraw(mol,fragcols,fragbcols)
    reassignTransparencies(tval)
    return ret